In [4]:
%pip install comet_ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.9/561.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.2/512.2 kB 20.4 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.6.2
    Uninstalling websocket-client-1.6.2:
      Successfully uninstalled websocket-client-1.6.2
  Attempting uninstall: python-box
    Found existing installation: python-box 7.1.1
    Uninstalling python-box-7.1.1:
      Successfully uninstalled python-box-7.1.1


In [5]:
import comet_ml
comet_ml.init(project_name="Prueba comet")

Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)
Comet API key: ··········


COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


In [6]:
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint
import gzip
import pickle
import numpy as np

In [7]:
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
    log_code=True,
)

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: keras, tensorflow, tensorboard.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/eduardo-macavity/prueba-comet/9e7fd987be67436e9d81a26c64e4707c



In [9]:
modelo_cargado = tf.keras.models.load_model('best_model.hdf5')

In [10]:
#Cargamos los datos
f = gzip.open('mnist.pkl.gz', 'rb')
dataset = pickle.load(f,encoding="latin1")

In [11]:


dat=np.array(dataset)
print(dat[1,1].shape)
(x_train, y_train), (x_val, y_val), (x_test, y_test) = dataset



(10000,)


<ipython-input-11-57f718d27ce6>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dat=np.array(dataset)


In [12]:


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')



In [13]:

num_classes=10
y_trainc = keras.utils.to_categorical(y_train, num_classes)
y_testc = keras.utils.to_categorical(y_test, num_classes)

In [14]:


parameters = {
    "batch_size": 128,
    "epochs": 30,
    "optimizer": "adam",
    "loss": "binary_crossentropy",
}

experiment.log_parameters(parameters)

In [15]:
model = Sequential(modelo_cargado)
model.add(Dense(512, activation='sigmoid', input_shape=(784,)))
model.add(Dense(600,activation='tanh'))
model.add(Dense(200,activation='selu'))
model.add(Dense(600,activation='relu'))
model.add(Dropout(0.5)) #Fraccion de enlaces a eliminar
model.add(Dense(100,activation='elu'))
model.add(Dense(600, activation='sigmoid', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))) #Regularizacion L1L2
model.add(Dense(500,activation='exponential'))
model.add(Dense(num_classes, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (None, 10)                1376730   
                                                                 
 dense (Dense)               (None, 512)               5632      
                                                                 
 dense_1 (Dense)             (None, 600)               307800    
                                                                 
 dense_2 (Dense)             (None, 200)               120200    
                                                                 
 dense_3 (Dense)             (None, 600)               120600    
                                                                 
 dropout (Dropout)           (None, 600)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               6

In [16]:
model.compile(loss=parameters['loss'],optimizer=parameters['optimizer'],metrics=['accuracy'])


In [17]:
model.fit(x_train, y_trainc,
                    batch_size=parameters['batch_size'],
                    epochs=parameters["epochs"],
                    verbose=1,
                    validation_data=(x_test, y_testc),
                    )

Epoch 1/30
391/391 [==============================] - 33s 78ms/step - loss: 0.2941 - accuracy: 0.3233 - val_loss: 0.2642 - val_accuracy: 0.4797
Epoch 2/30
391/391 [==============================] - 27s 69ms/step - loss: 0.2625 - accuracy: 0.3294 - val_loss: 0.2901 - val_accuracy: 0.1931
Epoch 3/30
391/391 [==============================] - 27s 69ms/step - loss: 0.2961 - accuracy: 0.1862 - val_loss: 0.2769 - val_accuracy: 0.2680
Epoch 4/30
391/391 [==============================] - 27s 69ms/step - loss: 0.2479 - accuracy: 0.3291 - val_loss: 0.2329 - val_accuracy: 0.3707
Epoch 5/30
391/391 [==============================] - 31s 80ms/step - loss: 0.2778 - accuracy: 0.2526 - val_loss: 0.3046 - val_accuracy: 0.1724
Epoch 6/30
391/391 [==============================] - 27s 69ms/step - loss: 0.3018 - accuracy: 0.1808 - val_loss: 0.2928 - val_accuracy: 0.2103
Epoch 7/30
391/391 [==============================] - 27s 70ms/step - loss: 0.2917 - accuracy: 0.1912 - val_loss: 0.2962 - val_accuracy:

In [23]:
score = model.evaluate(x_test, y_testc, verbose=1) #evaluar la eficiencia del modelo
print(score)
a=model.predict(x_test) #predicción de la red entrenada
print(a.shape)
print(a[1])
print("resultado correcto:")
print(y_testc[1])


313/313 [==============================] - 4s 12ms/step - loss: 0.3252 - accuracy: 0.1010
[0.3251815736293793, 0.10100000351667404]
313/313 [==============================] - 3s 8ms/step
(10000, 10)
[0.09097644 0.11060568 0.10239404 0.11190856 0.0993561  0.09060562
 0.09981892 0.10577936 0.09582642 0.09937967]
resultado correcto:
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


In [24]:
experiment.log_model("MNIST1", "best_model.hdf5")

In [25]:
experiment.end()

In [21]:
#Para guardar el modelo en disco
#model.save("red.h5")

In [22]:
#para cargar la red:
#modelo_cargado = tf.keras.models.load_model('red.h5')